# Kodun Ai Veya İnsan Yazımı Olduğunu Tahmin Etme Algoritması

In [ ]:
import pandas as pd #gerekli kütüphaneleri yüklüyoruz.
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
    
from sklearn.model_selection import train_test_split

import pickle
import warnings
warnings.filterwarnings('ignore')  

## EDA - Exploratory Data Analysis - Keşifçi Veri Analizi

In [ ]:
df=pd.read_excel("python_ai_dataset_7000.xlsx") #ai verilerini ekliyoruz.
df

In [ ]:
df["code"][0]

In [ ]:
df2=pd.read_excel("github_python_filtered_7000_repos.xlsx") #İnsan verilerini ekliyoruz.
df2

In [ ]:
df2["Kod"][0]

In [8]:
df=df[["code"]]
df2=df2[["Kod"]]
df.head()

,code
0,import datetime\n\nclass DateTimeHelper:\n ...
1,"def merge_dictionaries(dict1: dict, dict2: dic..."
2,"try:\n num = int(input(""Enter a number: ""))\n..."
3,import re\n\nclass PhoneNumberMatcher:\n de...
4,import sqlite3\n\ndef connect_db(db_path):\n ...


In [9]:
df2.head()

,Kod
0,#!/usr/bin/env python\n#\n# Copyright 2006 Goo...
1,"__version__ = ""0.1-pre""\nfrom django.conf.urls..."
2,\nimport collections\nimport datetime\nimport ...
3,#\n# Newf: A New Python (web) Framework\n# ...
4,"from setuptools import setup, find_packages\n ..."


In [10]:
df=df.drop_duplicates(["code"]) #Tekrar eden verileri siliyoruz
df.shape

(5505, 1)

In [11]:
df2=df2.drop_duplicates(["Kod"])
df2.shape

(6711, 1)

In [12]:
df=df.sample(5500)
df.reset_index(drop=True, inplace=True) #Rastgele 5500 veriyi alıyoruz.
df

,code
0,import shutil\nimport os\nimport logging\n\nde...
1,from multiprocessing import Pool\ndef process_...
2,import seaborn as sns\nimport matplotlib.pyplo...
3,import shutil\nimport os\n\ndef move_file_bugg...
4,"import shutil\nimport os\n\nsource = ""my_file...."
...,...
5495,import pandas as pd\n\nclass DataFrameFilter:\...
5496,import logging\n\ndef configure_logger(log_fil...
5497,import os\n\ndef read_write_text_file(input_fi...
5498,import sqlite3\nimport os\nimport logging\n\nl...


In [13]:
df2=df2.sample(5500)
df2.reset_index(drop=True, inplace=True)
df2

,Kod
0,# -*- coding: utf-8 -*-\n\nimport logging\nimp...
1,#!/usr/bin/env python\n#coding=utf-8\n\nimport...
2,"""<Include a description of your project>""\n\n_..."
3,"# Licensed under the Apache License, Version 2..."
4,#-*- coding: utf8 -*-\nimport re\nimport os\nf...
...,...
5495,"setupdict= {\n 'name': 'txAMQP',\n 'vers..."
5496,# -*- coding: utf-8 -*-\n\nfrom numpy import *...
5497,"""""""Simple benchmark to compare the speed of Be..."
5498,"from models import Event, EventCategory\nfrom ..."


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5500 entries, 0 to 5499
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    5500 non-null   object
dtypes: object(1)
memory usage: 43.1+ KB


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5500 entries, 0 to 5499
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Kod     5500 non-null   object
dtypes: object(1)
memory usage: 43.1+ KB


In [16]:
df.isnull().sum() #Boş satır var mı diye kontrol ediyoruz.

code    0
dtype: int64

In [17]:
df2.isnull().sum()

Kod    0
dtype: int64

In [18]:
df["Person"] = False #Etiketleme işlemini yapıyoruz.
df2["Person"] = True
df

,code,Person
0,import shutil\nimport os\nimport logging\n\nde...,False
1,from multiprocessing import Pool\ndef process_...,False
2,import seaborn as sns\nimport matplotlib.pyplo...,False
3,import shutil\nimport os\n\ndef move_file_bugg...,False
4,"import shutil\nimport os\n\nsource = ""my_file....",False
...,...,...
5495,import pandas as pd\n\nclass DataFrameFilter:\...,False
5496,import logging\n\ndef configure_logger(log_fil...,False
5497,import os\n\ndef read_write_text_file(input_fi...,False
5498,import sqlite3\nimport os\nimport logging\n\nl...,False


In [19]:
df2

,Kod,Person
0,# -*- coding: utf-8 -*-\n\nimport logging\nimp...,True
1,#!/usr/bin/env python\n#coding=utf-8\n\nimport...,True
2,"""<Include a description of your project>""\n\n_...",True
3,"# Licensed under the Apache License, Version 2...",True
4,#-*- coding: utf8 -*-\nimport re\nimport os\nf...,True
...,...,...
5495,"setupdict= {\n 'name': 'txAMQP',\n 'vers...",True
5496,# -*- coding: utf-8 -*-\n\nfrom numpy import *...,True
5497,"""""""Simple benchmark to compare the speed of Be...",True
5498,"from models import Event, EventCategory\nfrom ...",True


In [20]:
df.rename(columns={"code": "Code"}, inplace=True)
df2.rename(columns={"Kod": "Code"}, inplace=True)
df = pd.concat([df, df2], axis=0, ignore_index=True) #İnsan ve ai verilerini birleştiriyoruz.
df

,Code,Person
0,import shutil\nimport os\nimport logging\n\nde...,False
1,from multiprocessing import Pool\ndef process_...,False
2,import seaborn as sns\nimport matplotlib.pyplo...,False
3,import shutil\nimport os\n\ndef move_file_bugg...,False
4,"import shutil\nimport os\n\nsource = ""my_file....",False
...,...,...
10995,"setupdict= {\n 'name': 'txAMQP',\n 'vers...",True
10996,# -*- coding: utf-8 -*-\n\nfrom numpy import *...,True
10997,"""""""Simple benchmark to compare the speed of Be...",True
10998,"from models import Event, EventCategory\nfrom ...",True


## Feature Engineering

In [22]:
df["Code"]=df["Code"].str.lower() #tüm harfleri küçültüyoruz.
df["Code"]=df["Code"].str.replace("[^\w\s]","") #Noktalama işaretlerini temizliyoruz.
df["Code"]=df["Code"].str.replace("\d+","") #Sayıları kaldırıyoruz
df["Code"]=df["Code"].str.replace("\n","").replace("\r","") #boşlukları vs temizliyoruz.

In [23]:
df

,Code,Person
0,import shutilimport osimport loggingdef move_f...,False
1,from multiprocessing import pooldef process_it...,False
2,import seaborn as snsimport matplotlib.pyplot ...,False
3,"import shutilimport osdef move_file_buggy(src,...",False
4,"import shutilimport ossource = ""my_file.txt""de...",False
...,...,...
10995,"setupdict= { 'name': 'txamqp', 'version'...",True
10996,# -*- coding: utf-8 -*-from numpy import *impo...,True
10997,"""""""simple benchmark to compare the speed of be...",True
10998,"from models import event, eventcategoryfrom dj...",True


## Machine Learning - Classification

In [25]:
vect=CountVectorizer(lowercase=True,stop_words="english") #Gereksiz kelimeleri temizleyip vektörize ediyoruz.
x=df.Code
y=df.Person
x=vect.fit_transform(x)

In [26]:
def nlp_classification_funct(x,y):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
    
    print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)
    
    k=KNeighborsClassifier()
    d=DecisionTreeClassifier()
    r=RandomForestClassifier()
    l=LogisticRegression()
    mb=MultinomialNB()
    
    algos=[k,d,r,l,mb]
    algo_names=['KNeighborsClassifier','DecisionTreeClassifier','RandomForestClassifier','LogisticRegression',
                'MultinomialNB']
    
    accuracy_scored=[]
    precision_scored=[]
    recall_scored=[]
    f1_scored=[]
    
    for item in algos:
        item.fit(x_train,y_train)
        accuracy_scored.append(accuracy_score(y_test,item.predict(x_test)))
        precision_scored.append(precision_score(y_test,item.predict(x_test)))
        recall_scored.append(recall_score(y_test,item.predict(x_test)))
        f1_scored.append(f1_score(y_test,item.predict(x_test)))
        
    result=pd.DataFrame(columns=['f1_score','recall_score','precision_score','accuracy_score'],index=algo_names)
    result.f1_score=f1_scored
    result.recall_score=recall_scored
    result.precision_score=precision_scored
    result.accuracy_score=accuracy_scored
    nlp_classification_funct.result=result.sort_values('f1_score',ascending=False)
    return result.sort_values('f1_score',ascending=False)

In [27]:
nlp_classification_funct(x,y) #tahminleri yapıyoruz

(8800, 399349) (2200, 399349) (8800,) (2200,)


,f1_score,recall_score,precision_score,accuracy_score
RandomForestClassifier,0.992620,0.985348,1.000000,0.992727
LogisticRegression,0.990766,0.982601,0.999069,0.990909
DecisionTreeClassifier,0.990389,0.990842,0.989936,0.990455
MultinomialNB,0.971583,0.986264,0.957333,0.971364
KNeighborsClassifier,0.866044,0.763736,1.000000,0.882727


In [28]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
d=DecisionTreeClassifier()
r=RandomForestClassifier()
l=LogisticRegression()
d.fit(x_train,y_train)
r.fit(x_train,y_train)
l.fit(x_train,y_train)
print(f1_score(y_test,d.predict(x_test)))
print(f1_score(y_test,r.predict(x_test)))
print(f1_score(y_test,l.predict(x_test)))

0.9880843263061412
0.9935483870967742
0.9907663896583564


In [29]:
pickle.dump(d,open("DecisionTreeClassifier.pkl","wb"))
pickle.dump(r,open("RandomForestClassifier.pkl","wb"))
pickle.dump(l,open("LogisticRegression.pkl","wb"))
pickle.dump(vect, open("vectorizer.pkl", "wb")) #modelleri kaydediyoruz.

In [ ]:
import os, glob
print("CWD:", os.getcwd())
print("PKL here:", glob.glob("*.pkl"))
print("PKL recursive:", glob.glob("**/*.pkl", recursive=True)[:50])
